<a href="https://colab.research.google.com/github/ppfenninger/Sensorimotor_Learning_Final/blob/main/Sensorimotor_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Installation
!pip install pybullet > /dev/null 2>&1
!pip install git+https://github.com/taochenshh/easyrl.git > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install git+https://github.com/ppfenninger/airobot.git > /dev/null 2>&1
# !pip install git+https://github.com/ppfenninger/Sensorimotor_Learning_Final.git > /dev/null 2>&1

In [2]:
import os
import torch
import gym
import pprint
import time
import pybullet as p
import pybullet_data as pd
import pybullet_envs
import airobot as ar
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from typing import Any
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pylab
from dataclasses import dataclass
from gym import spaces
from gym.envs.registration import registry, register
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm.notebook import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from copy import deepcopy
from itertools import count
from easyrl.agents.ppo_agent import PPOAgent
from easyrl.utils.common import save_traj
from easyrl.configs import cfg
from easyrl.configs import set_config
from easyrl.configs.basic_config import BasicConfig
from easyrl.configs.command_line import cfg_from_cmd
from easyrl.engine.ppo_engine import PPOEngine
from easyrl.models.categorical_policy import CategoricalPolicy
from easyrl.models.diag_gaussian_policy import DiagGaussianPolicy
from easyrl.models.mlp import MLP
from easyrl.models.value_net import ValueNet
from easyrl.agents.base_agent import BaseAgent
from easyrl.utils.torch_util import DictDataset
from easyrl.utils.torch_util import load_state_dict
from easyrl.utils.torch_util import load_torch_model
from easyrl.runner.nstep_runner import EpisodicRunner
from easyrl.utils.torch_util import save_model
from easyrl.utils.torch_util import action_entropy
from easyrl.utils.torch_util import action_from_dist
from easyrl.utils.torch_util import action_log_prob
from easyrl.utils.torch_util import clip_grad
from easyrl.utils.common import set_random_seed
from easyrl.utils.gym_util import make_vec_env
from easyrl.utils.common import load_from_json
from easyrl.utils.torch_util import freeze_model
from easyrl.utils.torch_util import move_to
from easyrl.utils.torch_util import torch_float
from easyrl.utils.torch_util import torch_to_np
from base64 import b64encode
from IPython import display as ipythondisplay

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/easyrl/envs/shmem_vec_env.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool: ctypes.c_bool}


In [3]:
# del sys.modules["de_agent"]
# del sys.modules["de_runner"]
# del sys.modules["utils"]
# del sys.modules["de_env"]
# del sys.modules["de_engine"]

# install our library
!rm Sensorimotor_Learning_Final -r
!git clone -b testing https://github.com/ppfenninger/Sensorimotor_Learning_Final.git
import sys
sys.path.insert(0, './Sensorimotor_Learning_Final/deepexploration/')
import de_agent
from de_agent import DeepExplorationAgent
# import de_engine
import de_runner # this should work once everything compiles
from de_runner import DeepExplorationRunner
import utils
from utils import eval_agent, load_expert_agent, create_actor, create_critic

import de_env
from de_env import URRobotPusherGym

import de_engine
from de_engine import DeepExplorationEngine

module_name = __name__

env_name = 'URPusher-v1'
if env_name in registry.env_specs:
    del registry.env_specs[env_name]
register(
    id=env_name,
    entry_point=f'{module_name}:URRobotPusherGym',
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'Sensorimotor_Learning_Final'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 512 (delta 143), reused 113 (delta 82), pack-reused 332
Receiving objects: 100% (512/512), 160.67 KiB | 13.39 MiB/s, done.
Resolving deltas: 100% (324/324), done.


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(


# Setup

In [4]:
class DEConfig(BasicConfig):
    num_traj = 1

def set_configs(exp_name='de'):
    cfg.alg = DEConfig()
    cfg.alg.seed = 2 #seed
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 150
    cfg.alg.max_steps = 600000
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = 'URPusher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += f'/{exp_name}'
    cfg.alg.policy_lr: float = 3e-4
    cfg.alg.value_lr: float = 1e-3
    cfg.alg.linear_decay_lr: bool = False
    cfg.alg.max_decay_steps: int = 1e6
    cfg.alg.num_envs: int = 8
    cfg.alg.eval_num_envs: int = None
    cfg.alg.opt_epochs: int = 10
    cfg.alg.normalize_adv: bool = True
    cfg.alg.clip_vf_loss: bool = False
    cfg.alg.vf_loss_type: str = 'mse'
    cfg.alg.vf_coef: float = 0.5
    cfg.alg.ent_coef: float = 0.01
    cfg.alg.clip_range: float = 0.2
    cfg.alg.linear_decay_clip_range: bool = False
    cfg.alg.gae_lambda: float = 0.95
    cfg.alg.rew_discount: float = 0.99
    cfg.alg.use_amsgrad: bool = True
    cfg.alg.sgd: bool = False
    cfg.alg.momentum: float = 0.00
    cfg.alg.tanh_on_dist: bool = False
    cfg.alg.std_cond_in: bool = False
    cfg.alg.log_interval = 14
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'====================================')

In [5]:
# Runner Tests
set_configs()
env = URRobotPusherGym()

critics = []
for index in range(4):
  ob_size = env.observation_space.shape[0]
  critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
  critic = ValueNet(critic_body, in_features=64)
  critics.append(critic)

actor = create_actor(env=env)
agent = DeepExplorationAgent(actor=actor, critics=critics, env=env)
runner = DeepExplorationRunner(agent=agent, env=env)

# traj = runner(time_steps=cfg.alg.episode_steps)

[INFO][2023-05-06 23:20:38]: Load in OpenGL!
INFO:AIRobot:Load in OpenGL!


      Device:cuda


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


i have this many people telling me what to do 4


In [6]:
def train_de(max_steps=200000):
    set_configs()
    cfg.alg.num_envs = 1
    cfg.alg.episode_steps = 100
    cfg.alg.max_steps = max_steps
    cfg.alg.deque_size = 20
    cfg.alg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    cfg.alg.env_name = 'URPusher-v1'
    cfg.alg.save_dir = Path.cwd().absolute().joinpath('data').as_posix()
    cfg.alg.save_dir += '/'
    cfg.alg.save_dir += 'push'
    setattr(cfg.alg, 'diff_cfg', dict(save_dir=cfg.alg.save_dir))

    print(f'====================================')
    print(f'      Device:{cfg.alg.device}')
    print(f'      Total number of steps:{cfg.alg.max_steps}')
    print(f'====================================')

    set_random_seed(cfg.alg.seed)
    env_kwargs=dict()
    env = make_vec_env(cfg.alg.env_name,
                       cfg.alg.num_envs,
                       seed=cfg.alg.seed,
                       env_kwargs=env_kwargs)
    env.reset()
    ob_size = env.observation_space.shape[0]

    actor_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)

    critic_body = MLP(input_size=ob_size,
                     hidden_sizes=[64],
                     output_size=64,
                     hidden_act=nn.Tanh,
                     output_act=nn.Tanh)
  
    if isinstance(env.action_space, gym.spaces.Discrete):
        act_size = env.action_space.n
        actor = CategoricalPolicy(actor_body,
                                 in_features=64,
                                 action_dim=act_size)
        
    elif isinstance(env.action_space, gym.spaces.Box):
        act_size = env.action_space.shape[0]
        actor = DiagGaussianPolicy(actor_body,
                                   in_features=64,
                                   action_dim=act_size,
                                   tanh_on_dist=cfg.alg.tanh_on_dist,
                                   std_cond_in=cfg.alg.std_cond_in)
    else:
        raise TypeError(f'Unknown action space type: {env.action_space}')

    critics = [] # ValueNet(critic_body, in_features=64) # TODO: get critics

    for critic in range(2):
      critics.append(create_critic(env))


    agent = DeepExplorationAgent(actor=actor, critics=critics, env=env)
    runner = DeepExplorationRunner(agent=agent, env=env)
    engine = DeepExplorationEngine(agent=agent, runner=runner, env=env)
    engine.train()
    stat_info, raw_traj_info = engine.eval(render=False, save_eval_traj=True, eval_num=1, sleep_time=0.0)
    pprint.pprint(stat_info)
    return cfg.alg.save_dir

# Testing

In [ ]:
saved_dir_pusher = train_de(max_steps = 1000)

[INFO][2023-05-06 23:20:43]: Creating 1 environments.
INFO:EasyRL:Creating 1 environments.
[INFO][2023-05-06 23:20:43]: Load in OpenGL!
INFO:AIRobot:Load in OpenGL!


      Device:cuda
      Device:cuda
      Total number of steps:1000


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/easyrl/envs/dummy_vec_env.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning

i have this many people telling me what to do 2
iter: 0
started eval agent


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
[INFO][2023-05-06 23:20:45]: Exploration steps: 0
INFO:EasyRL:Exploration steps: 0
[INFO][2023-05-06 23:20:45]: Saving checkpoint: /content/data/push/seed_2/model/ckpt_000000000000.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2/model/ckpt_000000000000.pt.


finished eval agent
time_steps 100
random_action False
exploration_enabled True
We are done! 29
time_steps 100
random_action False
exploration_enabled True
We are done! 29


[INFO][2023-05-06 23:20:45]: Saving checkpoint: /content/data/push/seed_2/model/model_best.pt.
INFO:EasyRL:Saving checkpoint: /content/data/push/seed_2/model/model_best.pt.


1
time_steps 100
random_action False
exploration_enabled True
2
8
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
9
3
iter: 1
1
time_steps 100
random_action False
exploration_enabled True
2
8
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
9
3
iter: 2
1
time_steps 100
random_action False
exploration_enabled True
2
8
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
200
100
a
b
c
d
e
f
g
h
i
20